In [ ]:
# 使用MindFormers套件在mindspore环境下对LLaMA 进行LoRA微调
# 基座模型：LLaMA3-8B

# 首先进行数据集及权重准备
# 数据集采用 PsyQA_full.json，需要先将 PsyQA_full 数据集转换为 alpaca_data.json 数据集的格式
# 再使用fastchat工具添加prompts模板，将PsyQA转换为多轮对话格式。

python ../mindformers/mindformers/tools/dataset_preprocess/llama/alpaca_converter.py \
  --data_path ../dataset/PsyQA.json \
  --output_path ../dataset/PsyQA-data-conversation.json

In [ ]:
# 生成数据集对应的Mindrecord数据，将带有prompt模板的数据转换为mindrecord格式。
python ../mindformers/research/llama3/llama3_preprocess.py \
  --dataset_type qa \
  --input_glob ../dataset/PsyQA-data-conversation.json \
  --model_file ../mindformers/tokenizer.model \
  --seq_length 8192 \
  --output_file ../dataset/PsyQA.mindrecord

In [ ]:
# 模型权重转换，将huggingface的权重转换为完整的ckpt权重。
# input_path:  下载HuggingFace权重的文件夹路径
# output_path: 转换后的MindSpore权重文件保存路径

python ../mindformers/convert_weight.py --model llama --input_path ../Meta-Llama-3-8B --output_path ../mindformers/research/llama3/model_dir --dtype bf16

In [ ]:
# 全参微调SFT
# 单机8卡训练
bash ../mindformers/scripts/msrun_launcher.sh "run_mindformer.py \
 --register_path ../mindformers/research/llama3 \
 --config ../mindformers/research/llama3/llama3_8b/finetune_llama3_8b.yaml \
 --load_checkpoint ../mindformers/research/llama3/model_dir \
 --auto_trans_ckpt True \
 --use_parallel True \
 --run_mode finetune \
 --train_data ../dataset/PsyQA.mindrecord"

In [ ]:
# 单卡推理
python ../mindformers/run_mindformer.py \
 --register_path ../mindformers/research/llama3 \
 --config ../mindformers/research/llama3/llama3_8b/predict_llama3_8b.yaml \
 --load_checkpoint ../mindformers/research/llama3/model_dir \
 --auto_trans_ckpt False \
 --use_parallel False \
 --run_mode predict \
 --predict_data "你好，我想进行抑郁筛查"